# Example: Grocery Store Fruit Problem using a Linear Utility Function
This example will familiarize students with solving constrained maximum utility problems and will serve as an introduction to linear programming. In this example, we'll use a numerical approach to solve for the optimal allocation between `apples` and `oranges,` assuming a `Linear` utility function. In particular,
* We'll estimate the optimal case we expect, solve the linear programming problem, and compare the solutions 
* We'll visualize the budget constraint and the indifference curve and show that the numerical optimum is either a corner solution or the entire resource constraint line

### Problem
Consider a utility maximization problem that uses a Linear utility function $U(x_{1},x_{2}) = \alpha_{1}x_{1}+\alpha_{2}x_{2}$ subject to a budget constraint. The Lagrangian is:
$$
\begin{equation*}
\mathcal{L}(x,\lambda) = \alpha_{1}x_{1} + \alpha_{2}x_{2} + \lambda\cdot\left(I-c_{1}x_{1}-c_{2}x_{2}\right)
\end{equation*}
$$
which gives the first-order optimality conditions:
$$
\begin{eqnarray*}
\frac{\partial\mathcal{L}}{\partial{x_{1}}} & = & \alpha_{1} - \lambda\cdot{c_{1}} = 0 \\
\frac{\partial\mathcal{L}}{\partial{x_{2}}} & = & \alpha_{2} - \lambda\cdot{c_{2}} = 0 \\
\frac{\partial\mathcal{L}}{\partial\lambda} & = & I - c_{1}x_{1} - c_{2}x_{2} = 0
\end{eqnarray*}
$$

## Setup
The computations in this lab (or example) are enabled by the [VLDecisionsPackage.jl](https://github.com/varnerlab/VLDecisionsPackage.jl.git) and several external `Julia` packages. To load the required packages and any custom codes the teaching team has developed to work with these packages, we [include](https://docs.julialang.org/en/v1/manual/code-loading/) the `Include.jl` file):

In [3]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
   Resolving package versions...
    Updating `~/Desktop/julia_work/CHEME-4800-5800-Examples-Fall-2024/lecture/week-9/L9a/Project.toml`
  [5ae59095] + Colors v0.12.11
  [a93c6f00] + DataFrames v1.7.0
  [60bf3e95] + GLPK v1.2.1
  [4076af6c] + JuMP v1.23.3
  [91a5bcdd] + Plots v1.40.8
  [10f378ab] + VLDecisionsPackage v0.1.0 `https://github.com/varnerlab/VLDecisionsPackage.jl.git#main`
  [37e2e46d] ~ LinearAlgebra ⇒ v1.11.0
    Updating `~/Desktop/julia_work/CHEME-4800-5800-Examples-Fall-2024/lecture/week-9/L9a/Manifest.toml`
  [79e6a3ab] + Adapt v4.0.4
  [66dad0bd] + AliasTables v1.1.3
  [4fba245c] + ArrayInterface v7.16.0
  [6e4b80f9] + BenchmarkTools v1.5.0
  [d1d4a3ce] + BitFlags v0.1.9
  [523fee87] + CodecBzip2 v0.8.4
  [944b1d66] + CodecZlib v0.7.6
  [35d6a980] + ColorSchemes v3.26.0
⌅ [3da002f7] + ColorTypes v0.11.5
  [c3611d14] + ColorVectorSpace v0.10.0
  [5ae59095] + Colors v0.12.11
  [bbf7d656] + Co

## Prerequisites
Let's set the prices, $\alpha_{i}$, and budget for our `Apple` versus `Oranges` problem. We'll store the prices in the `c` array and the coefficients in the utility function in the `α` variable.
* The $\alpha_{i}$ coefficients (because we have a linear utility function) are the [marginal utilities](https://en.wikipedia.org/wiki/Marginal_utility), i.e., they tell us the satisfaction we gain from consuming an additional unit of good $i$. They have units of `utils/qty`
* The $c_{i}$ coefficients represent the unit cost of each good, e.g., the cost of a single apple or orange. The $c_{i}$ coefficients have units of `USD/qty.`
* Finally, the `total_budget` variable holds the amount of money we spend on apples and oranges. The `total_budget` has units of `USD`.

In [5]:
# α = [0.55, 0.45]; # coefficients for case A
α = [0.15, 0.55]; # coefficients for case B
# α = [2.0, 4.0]; # coefficients for case C
c = [2.0 4.0]; # price of x1 and x2
total_budget = 100.0;

### Check: Are the first-order optimality LAEs underdetermined?
At first glance, it's not obvious (at least to me) why the system of linear algebraic equations representing the first-order optimality conditions should be underdetermined. Let's check this out by setting up the system matrix and computing the rank assuming $(x_{1},x_{2},\lambda)$:

In [7]:
A = [
    0 0 -c[1]     ; # x1 constraint
    0 0 -c[2]     ; # x2 contraint
    -c[1] -c[2] 0 ; # lambda constrain
];

In [8]:
(U,S,V) = svd(A);

In [9]:
S

3-element Vector{Float64}:
  4.47213595499958
  4.47213595499958
 -0.0

In [10]:
Â = let
    r = rank(A);
    M = copy(A) |> x-> fill!(x, 0.0); 

    # update -
    for i ∈ 1:r
        M += S[i]*⊗(U[:,i],V[:,i]);
    end
    M
end

3×3 Matrix{Float64}:
  0.0   0.0  -2.0
  0.0   0.0  -4.0
 -2.0  -4.0   0.0

## Task 1: Compute the optimal A/O mix using a Linear Program
Hmmm. So the [Lagrange multiplier method](https://en.wikipedia.org/wiki/Lagrange_multiplier#:~:text=In%20mathematical%20optimization%2C%20the%20method,chosen%20values%20of%20the%20variables).) failed. Let's solve this as a [Linear programming problem](https://en.wikipedia.org/wiki/Linear_programming). Let's not worry so much about how a linear programming solver works (we are invoking the `buy` side of `buy` versus `build`). Instead, let's try to understand how the problem works, and then we'll dig into the details later. 

Here's how we compute the optimal value for a constrained `Linear` utility maximization problem. 
* First, we create an instance of the `MySimpleLinearChoiceProblem` type using the `build(...)` method. We input problem parameters, such as the utility function coefficients, cost values, total budget values, an initial guess, and allocation bounds.
* Next, we use the `solve(...)` method with the `problem` variable and store the solution in the `solution` variable.
* __What's up with the try-catch?__ The call to [the `solve(...)` method](src/Solve.jl) has a check to see if the solution returned by the solver is feasible. This check uses [the @assert macro](), which can throw [an AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError). So, we wrap our call to [the `solve(...)` method](src/Solve.jl) in [try-catch block](https://docs.julialang.org/en/v1/manual/control-flow/#The-try/catch-statement) to catch this exception gracefully should it occur.

In [12]:
# bounds on the choice variables -
bounds = [
    0.0 100.0; # L U
    0.0 100.0; # L U
]

# build my problem object -
problem = build(MyLinearProgrammingProblemModel, (
    
    initial = 0.1*ones(2), # initial guess for how  much x₁ and x₂ we are buying
    c = α, # coefficients
    A = c, # prices of x1 and x2
    b = [total_budget], # income
    
    # how much of x₁ and x₂ can be we buy?
    lb = bounds[:,1],
    ub = bounds[:,2]
));

solution = nothing;
try 
    # call the solve function. This will return a dictionary -
    solution = solve(problem);
catch error
    println(error); # what happned?
end

Dict{String, Any} with 2 entries:
  "argmax"          => [0.0, 25.0]
  "objective_value" => 13.75

The `solution` variable is a [Julia Dictionary type](https://docs.julialang.org/en/v1/base/collections/#Base.Dict) which holds three `string keys`:
* `argmax`: Holds the optimal solution vector
* `objective_value`: Holds the optimal value of the utility

In [14]:
solution

Dict{String, Any} with 2 entries:
  "argmax"          => [0.0, 25.0]
  "objective_value" => 13.75

### Check: What case do we have?
Depending upon the absolute value of the slope of the budget line $|m_{B}|$ and the indifference curve $|m_{I}|$, we should get one of three possible solution types:
* __Case A__: The `abs` slope of the indifference curve is _greater than_ the budget line, i.e., $ |m_{I}|>|m_{B}|$. This gives the `bottom` corner as a solution, i.e., we spend all our budget on good 1, in this case, `apples.`
* __Case B__: The `abs` slope of the indifference curve is _less than_ than the budget line, i.e., $|m_{I}|<|m_{B}|$. This gives the `top` corner as a solution, i.e., we spend all our budget on good 2, in this case, `oranges.`
* __Case C__: The `abs` slope of the indifference curve is _equal to_ the budget line, i.e., $|m_{I}|=|m_{B}|$. This gives the _any solution on the budget line_.

In [16]:
slope_budget_line = -1*(c[1]/c[2]);
slope_indifference_curve = -1*(α[1]/α[2])
println("slope budget = $(slope_budget_line), slope indiff = $(slope_indifference_curve)")

slope budget = -0.5, slope indiff = -0.2727272727272727


In [17]:
if (abs(slope_budget_line) < abs(slope_indifference_curve))
    println("Case A: Solution should be the x-intercept corner point")
elseif (abs(slope_budget_line) > abs(slope_indifference_curve))
    println("Case B: Solution should be the y-intercept corner point")
else (abs(slope_budget_line) == abs(slope_indifference_curve))
    println("Case C: We will have an infinite number of solutions")
end

Case B: Solution should be the y-intercept corner point


In [18]:
x̂₁ = total_budget/c[1]
x̂₂ = total_budget/c[2]
println("The corner solutions (apples,oranges) = ($(x̂₁),$(x̂₂))")

The corner solutions (apples,oranges) = (50.0,25.0)


#### Check: Does this solution satisfy the first-order optimality conditions?
The solution calculated by the linear program should satisfy at most rank `r` number of first-order optimality conditions. Let's check this out by grabbing the calculated solution and various components of the problem and computing the optimality conditions. 
* We'll use the [isapprox function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) function to check how close each optimality equation is away from `0` where we set `atol` parameter to control what we mean by `close`. The [isapprox function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) returns a `Bool` value.

In [20]:
# get the solution values from the solution dictionary
x̂₁ = solution["argmax"][1];
x̂₂ = solution["argmax"][2];
c₁ = c[1];
c₂ = c[2];
I = dot(α, solution["argmax"]); # how much did we actually spend?

# compute the marginal utility at the optimum -
Ū₁ = α[1];
Ū₂ = α[2];

# compute the Lagrange multiplier (from eqn 1)
λ = Ū₁/c₁;
#λ = Ū₂/c₂;

# Compute the first-order conditions -
E1 = Ū₁ - λ*c₁
E2 = Ū₂ - λ*c₂
E3 = I - (c₁*x̂₁ + c₂*x̂₂)

# checkme -
isapprox.((E1,E2,E3),(0.0,0.0,0.0), atol=1e-10) # notice the . 

(true, false, false)

In [21]:
E1, E2, E3

(0.0, 0.25000000000000006, -86.25)

## Visualize the Resource line, Indifference curve, and the Optimal Solution
`Unhide` the code block below to see how we visualized the resource constraint, the optimal indifference curve, and the optimal solution.

In [ ]:
let
    # ok, let's grab the results dictionary and the Uopt -
    Uopt = round(solution["objective_value"], sigdigits=4);
    xopt = solution["argmax"];
    c = problem.c;
    budget_value = total_budget
    
    # indifference_bounds -
    indifference_bounds = [
        0.0 1.1*(budget_value/c[1]) ;
        0.0 1.1*(budget_value/c[2]) ;
    ];
    
    xlim = [0.0, 1.1*(budget_value/c[1])];
    ylim = [0.0, 1.1*(budget_value/c[2])];
    x1_opt = round(xopt[1]; sigdigits=3)
    x2_opt = round(xopt[2]; sigdigits=3)
    
    # compute the indifference curve -
    INC = indifference(problem, Uopt, indifference_bounds)
    
    # compute the budget -
    BC = budget(problem, xlim, total_budget)
    
    # make the plots -
    plot(INC[:,1],INC[:,2],label="", lw=3, c=:blue,
        bg="floralwhite", background_color_outside="white", framestyle = :box, fg_legend = :transparent);
    plot!(BC[:,1],BC[:,2], label="Budget $(budget_value) USD", lw=3,c=:black, xlim = xlim, ylim = ylim)
    scatter!([xopt[1]], [xopt[2]], ms=4, c=:orange, msc=:orange, label="Opt: ($(x1_opt),$(x2_opt))")
    xlabel!("Number of Apples", fontsize=18)
    ylabel!("Number of Oranges", fontsize=18)
end